In [99]:
# -*- coding:utf-8 -*- 

dic = {('a',1):100, ('b',3):"yes", ('c',10):0.98} 
print "1",dic.get('d')
#print dic['d']  沒d,所以會出錯
dic[('d',55)] = 250 
print "2",dic 
dic.update({('e',22):123, ('f',6):789}) 
#dic {'a': 100, 'c': 0.98, 'b': 'yes', 'e': 123, 'd': 'new', 'f': 789}
print "3",sorted(dic,key = lambda x : x[1]) #只能針對key排序與顯示
for rec in sorted(dic,key = lambda x : x[1]): 
    print rec, dic[rec]

1 None
2 {('a', 1): 100, ('d', 55): 250, ('b', 3): 'yes', ('c', 10): 0.98}
3 [('a', 1), ('b', 3), ('f', 6), ('c', 10), ('e', 22), ('d', 55)]
('a', 1) 100
('b', 3) yes
('f', 6) 789
('c', 10) 0.98
('e', 22) 123
('d', 55) 250
<type 'list'>


In [2]:
import requests
from bs4 import BeautifulSoup
import re
rs = requests.session()  

cookie = 'ASP.NET_SessionId=yriunnj3ftw1jvzg2o4bddid; lawbank=fdea33a84ca2489eb88bf7a7d24247b9-39; y=p=p0aZ1pxq302qP8oLU%2fV9F2e0UugVHGlWaEDpbkvUwrdN4kDY5j05nQCC03i0IHkoIVOSxKfcWmMzAhH%2bfKz7sA%3d%3d; _ga=GA1.3.816590347.1437135848; _gat=1; nrhbne=AgZLAZCvQeWFcxGAcpgsiQ==; JubFrm-pagebox=%5EcourtFullName%3DTPDV%60SLDV%60PCDV%60ILDV%60KLDV%60TYDV%60SCDV%60MLDV%60TCDV%60CHDV%60NTDV%60ULDV%60CYDV%60TNDV%60KSDV%60HLDV%60TTDV%60PTDV%60PHDV%60KMDV%60LCDV%60KSYV%60TPCM%60%5Ejud_case%3D%E5%A9%9A%5Ejcatagory%3D2%5Eissimple%3D-1%5Ejt%3D%E9%9B%A2%E5%A9%9A; x=i=qO/Zvq+XySl6717OeuzNDcU3Q5FCjdRzsmPT0LSay+LYmYSV9MKeyHr5B69ezAYZ&j=ExKZIgGGiQD19KUxjSulmRd4cCS013mf16VUoBCxWainbmYufAN3bzZLoLqLFhwxOnTKxyGFWxF875EJIi2dLxJH6hbqBxjAtbFdhUFXhN0N52w88aupjAgf+1xanz35gqeqSqCyVcutjKPBYXx2S4NWdfslegbdREC9AfMjAm+zkZefg3LaDBQlB1tXv4gpeRNEF+2/vUrprNHpdwTTB7ldH0fldmjQzRJ40RhXbZas97JDbl8rytgzE7yXF3pU1aOS+CB+FuM2RsvvSTyLiB4yU0fVo2DcpdJ56zuq9FI5QRN4h3epK1wnp517TGy1'
header={'Cookie':cookie}

local_diction={
    '臺灣台北地方法院':1,
    '臺灣士林地方法院':2,
    '臺灣新北地方法院':3,
    '臺灣宜蘭地方法院':4,
    '臺灣基隆地方法院':5,
    '臺灣桃園地方法院':6,
    '臺灣新竹地方法院':7,
    '臺灣苗栗地方法院':8,
    '臺灣臺中地方法院':9,
    '臺灣彰化地方法院':10,
    '臺灣南投地方法院':11,
    '臺灣雲林地方法院':12,
    '臺灣嘉義地方法院':13,
    '臺灣臺南地方法院':14,
    '臺灣高雄地方法院':15,
    '臺灣花蓮地方法院':16,
    '臺灣臺東地方法院':17,
    '臺灣屏東地方法院':18,
    '臺灣澎湖地方法院':19,
    '臺灣金門地方法院':20,
    '臺灣連江地方法院':21,
    '臺灣高雄少年及家事法院地方法院':22
    }    

res = requests.get('http://fyjud.lawbank.com.tw/listcontent5.aspx', headers=header) #只抓的到第一頁
#res.encoding #查看網頁編碼
soup = BeautifulSoup(res.text)
linklist = []
filename = []
print soup.select('.page td')[0].text  #頁數
for ele in soup.select('.Table-List td a'):
    linklist.append(ele.attrs['href'])
    
for list_num in range(0,len(linklist)-1):
    rs_get =rs.get('http://fyjud.lawbank.com.tw/'+linklist[list_num],headers=header)
    response = rs_get.text.encode('utf8')
    response_soup = BeautifulSoup(response)
    '''
    test_str=response_soup.select('.Table-List td')[1].text
    print test_str
    print type(test_str)
    print dir(test_str)
    '''
    filenumber=response_soup.select('.Table-List td')[1].text
    fileid=filenumber.split(',')[0].encode('utf8')+'05'+filenumber.split(',')[2].encode('utf8')
    localation = soup.select('h4')[0].text #臺灣台北地方法院 裁判書 -- 民事類
    local_list = re.match(r'(.+?) ', localation) #臺灣台北地方法院
    local_str =','.join(local_list.groups(0))
    local_id=local_diction[local_str.encode('utf8')]
    
    filename.append('sample'+str(local_id)+'_'+fileid)
    judgedate=response_soup.select('.Table-List td')[3].text.encode('utf8') #【裁判日期】1040630
    judge_reason=response_soup.select('.Table-List td')[5].text.encode('utf8') #【裁判案由】離婚
    content=response_soup.select('.Table-List td')[8].text.encode('utf8')#【裁判全文】
    record=fileid+";\n\n"+judgedate+";\n\n"+judge_reason+";\n\n"+content
    f = open('C:\Users\BigData\Desktop\DataSet\sample'+str(local_id)+'_'+\
             fileid+'.txt', 'w')
    f.writelines(record)
    f.close()

'''
104,家陸許,15
104,婚,108
104,家陸許,14
104,婚,216
104,婚,207
......
'''

共 3567 筆 第 1 / 179 頁


'\n104,\xe5\xae\xb6\xe9\x99\xb8\xe8\xa8\xb1,15\n104,\xe5\xa9\x9a,108\n104,\xe5\xae\xb6\xe9\x99\xb8\xe8\xa8\xb1,14\n104,\xe5\xa9\x9a,216\n104,\xe5\xa9\x9a,207\n......\n'

In [47]:
'''
for ele in filename:
    print ele
print 2,filename[0]  #第一個filename檔案
'''

import os
for dirPath, dirNames, fileNames in os.walk("C:\Users\BigData\Desktop\DataSet"):
    #print dirPath
    for f in fileNames: #fileNames是list
        #print f.encode('utf8') #f是str
        #print f.decode('utf8')
        #print ",".join(list(f))
        #for ele in list(f):
            #print ele
        #print fpath.encode('utf-8')

IndentationError: expected an indented block (<ipython-input-47-30ae90dff899>, line 18)

In [115]:
import re
a = soup.select('h4')[0].text #臺灣台北地方法院 裁判書 -- 民事類
m = re.match(r'(.+?) ', a) #臺灣台北地方法院
mm =','.join(m.groups(0))

In [33]:
#coding=utf-8
import jieba
import jieba.posseg as pseg
import operator

#list直接印列表中文部分會印出unicode編碼，必須用迴圈印才能印出正常編碼，因為list類別python預設轉為unicode
#方便程式內溝通

'''
for obj in filename:
    print obj
'''

fileword=""
dic = {}
result = []
jieba.set_dictionary("E:\iii\jieba\dict.txt.big.txt")


#for obj in filename:
    #str_convert = ''.join(obj.encode('utf-8'))
    #file_list = open('C:\\Users\\BigData\\Desktop\\DataSet\\'+ obj.decode('utf8'), 'r')
    #file_list_2= open('C:\\Users\\BigData\\Desktop\\DataSet\\'+ obj.decode('utf8')+'_2', 'w')
file_list = open('C:\\Users\\BigData\\Desktop\\DataSet\\'+ filename[0]+'.txt', 'r')
file_list_2= open('C:\\Users\\BigData\\Desktop\\DataSet\\'+ filename[0]+'.txt'+'_2', 'w')
#file_list = open('C:\\Users\\BigData\\Desktop\\DataSet\\sample103,婚,337.txt'.decode('utf8'), 'r')
#file_list_2= open('C:\\Users\\BigData\\Desktop\\DataSet\\sample103,婚,337_2.txt'.decode('utf8'), 'w')

for line in file_list.readlines():
    line_s=line.strip()
    result.append(line_s)
file_list_2.write(", ".join(result))
file_list_2.close()
file_list.close()

'''
for ele in result:
    print "TX"+ele
'''

result2 = ("".join(result))
#print result2
seg_list = jieba.cut_for_search(result2) #type 'generator'
#rint seg_list
print(", ".join(seg_list))
'''
words = pseg.cut(result2)
for word, flag in words:
    print('%s %s' % (word, flag))  #列印斷字與判斷出的詞性
'''
#for ele in seg_list:
    #print ele
#print type(seg_list) #generator
#print (",".join(seg_list))


for ele in list(seg_list):
    if ele not in dic:
        dic[ele] = 1
    else:
        dic[ele] = dic[ele] + 1
#print dic
#print seg_list[2]
#sorted_x = sorted(dic.items(), key=operator.itemgetter(1), reverse=True)
#sort對list排序；sorted對iterable排序  但排變list
#print sorted_x
'''
for ele in dic:
    print ele, dic[ele]
#直接印dic沒順序
'''

'''
for ele2 in sorted_x:
    #ele2 is tuple type
    print ele2  #編碼問題待解決
'''
    
#f = open('C:\Users\BigData\Desktop\DataSet\sample'+response_soup.select('.Table-List td')[1].text+'.txt', 'w')

Building prefix dict from E:\iii\jieba\dict.txt.big.txt ...
DEBUG:jieba:Building prefix dict from E:\iii\jieba\dict.txt.big.txt ...
Loading model from cache c:\users\bigdata\appdata\local\temp\jieba.uf77d3c66f86a246e288c1abbb15e06b0.cache
DEBUG:jieba:Loading model from cache c:\users\bigdata\appdata\local\temp\jieba.uf77d3c66f86a246e288c1abbb15e06b0.cache
Loading model cost 0.648 seconds.
DEBUG:jieba:Loading model cost 0.648 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


10405164, ;, 1040803, ;, 離婚, ;, 臺灣, 臺北, 地方, 方法, 法院, 地方法院, 民事, 判決, 民事判決, 　, 　, 　, 　, 　, 　, 　, 104, 年度, 婚字, 第, 164, 號原, 　, 　, 　, 告, 　, 張震儀, 訴訟, 代理, 代理人, 　, 鍾, 宛蓉, 律師, 被, 　, 　, 　, 告, 　, 高麗, , 上列, 當事, 當事人, 間, 請求, 離婚, 事件, ，, 本院, 於, 民國, 104, 年, 7, 月, 20, 日, 言詞, 辯論, 言詞辯論, 終結, ，, 判決, 如下, ：, 主,  ,  , 文准, 原告, 與, 被告, 離婚, 。, 訴訟, 訟費, 費用, 訴訟費, 訴訟費用, 由, 被告, 負擔, 。, 事實, 及, 理由, 一, 、, 被告, 經合, 法, 通知, 未, 於, 言詞, 辯論, 言詞辯論, 期, 日, 到場, ，, 核無, 民事, 訴訟, 訴訟法, 第, 386,  , 條, 各款, 所, 列, 情形, ，, 按照, 家事, 事件, 法, 第, 51, 條準, 用, 民事, 訴訟, 訴訟法, 第, 385, 條第, 1, 項, 前段, 規定, ，, 爰, 依, 原告, 之聲, 請, ，, 由, 其一, 造, 辯論, 而, 為, 判決, 。, 二, 、, 原告, 起訴, 主張, 略以, ：, 兩造, 前, 於, 民國, 91, 年, 在, 美國, 公證, 結婚, 公證結婚, 定居, 美國, ，, 復, 因, 被告, 父親, 生病, ，, 兩造, 即, 回國, 定居, ，, 就近, 照料, 。, 並於, 93, 年, 10, 月, 30, 日, 在, 國內, 舉行, 結婚, 儀式, 結婚儀式, 宴請, 賓客, ，, 嗣, 被告, 之, 父親, 於, 94, 年間, 病逝, ，, 被告, 執意, 爭取, 成為, 美國, 公民, ，, 而, 原告, 因在, 臺灣, 已有, 穩定, 工作, ，, 只好, 勉強, 同意, 被告, 單獨, 前往, 美國, ，, 兩造, 即, 自, 95, 年, 開始, 分居, 迄今, 長達, 八年, ，, 兩造, 分居, 期間, ，, 原告, 雙親, 及, 被告, 母親, 照顧, 責任, 及, 家庭, 經濟, 重擔, 皆, 由, 原告, 

'\nfor ele2 in sorted_x:\n    #ele2 is tuple type\n    print ele2  #\xe7\xb7\xa8\xe7\xa2\xbc\xe5\x95\x8f\xe9\xa1\x8c\xe5\xbe\x85\xe8\xa7\xa3\xe6\xb1\xba\n'

In [23]:
import jieba.analyse
jieba.analyse.set_idf_path("E:\iii\jieba\idf.txt.big") # file_name
jieba.analyse.set_stop_words("E:\iii\jieba\stop_words.txt") # file_name
keyword = jieba.analyse.extract_tags(result2, topK=20, withWeight=False, allowPOS=())  #此為list
print ",".join(keyword)

維持,夫妻,美國,難以,分居,生活,定居,民法,104,95,1052,八年,言詞辯論,20,破綻,父親,幸福,家事,10,要求


In [14]:
import jieba
sentences = ["我喜欢吃土豆","土豆是个百搭的东西","我不喜欢今天雾霾的北京", 'costumer service']
# jieba.suggest_freq(‘百搭‘, True)
words = [list(jieba.cut(doc)) for doc in sentences]
#print(words)
sen = ""
for ele in words:
    for ele2 in ele:
        sen = sen + ele2 + ','
print sen
#print(', '.join(words))
print jieba.suggest_freq('雾霾', True)

我,喜欢,吃,土豆,土豆,是,个,百,搭,的,东西,我,不,喜欢,今天,雾,霾,的,北京,costumer, ,service,
1


In [43]:
import jieba.analyse
sentence="我喜欢吃土豆,土豆是个百搭的东西,我不喜欢今天雾霾的北京"
tup=jieba.analyse.extract_tags(sentence, topK=1, withWeight=False, allowPOS=())
s = "".join(tup)
print(s)

土豆


In [5]:
# -*- coding: utf-8 -*-
'''
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time
import os

#browser = webdriver.Firefox()
#browser = webdriver.Ie('IEDriverServer.exe')
browser = webdriver.Chrome(executable_path=r'C:\Python27\WebDriver\chromedriver.exe')
browser.get('http://www.google.com.tw/')
elem = browser.find_element_by_xpath("//input[@id='lst-ib']")
elem.send_keys(u'艾小克' + Keys.RETURN)
time.sleep(1)
links = browser.find_elements_by_xpath("//h3[@class='r']/a")
assert links[0].text == u'瓶水相逢- 艾小克', 'not match'   ##有錯
browser.close()'''

from selenium import webdriver

driver = webdriver.Chrome(executable_path=r'C:\Python27\WebDriver\chromedriver.exe')
#driver.get('http://fyjud.lawbank.com.tw/listcontent5.aspx', headers=header)
#browser.find_element_by_id("su").click()
driver.get('http://www.baidu.com/')
print driver.title  # 把页面title 打印出来
driver.quit()

百度一下，你就知道


In [25]:
pwd

u'C:\\Users\\BigData'